<a href="https://colab.research.google.com/github/ZyroadGM/509900Saksas-na-Henri-R/blob/master/secureCom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import base64
from sympy import isprime, mod_inverse

def generate_prime(bits=512):
    """Generate a random prime number with the given bit length."""
    while True:
        num = random.getrandbits(bits)
        if isprime(num):
            return num

def asymmetricGenerateKeys(bits=512):
    """Generate RSA public and private keys."""
    p = generate_prime(bits)
    q = generate_prime(bits)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    e = 65537
    while e >= phi_n or phi_n % e == 0:
        e = random.randrange(2, phi_n)

    d = mod_inverse(e, phi_n)
    return ((e, n), (d, n))

def asymmetricEncrypt(message, public_key):
    """Encrypt a message and return a Base64-encoded string."""
    e, n = public_key
    m = int.from_bytes(message.encode(), 'big')  # Convert message to integer
    c = pow(m, e, n)  # RSA encryption
    return base64.b64encode(str(c).encode()).decode()  # Make it printable

def asymmetricDecrypt(ciphertext, private_key):
    """Decrypt a Base64-encoded ciphertext back to a message."""
    d, n = private_key
    try:
        c = int(base64.b64decode(ciphertext).decode())  # Decode from Base64
        m = pow(c, d, n)  # RSA decryption
        return m.to_bytes((m.bit_length() + 7) // 8, 'big').decode('utf-8')  # Convert to string
    except (UnicodeDecodeError, ValueError):
        print("Decryption failed: Invalid ciphertext or incorrect key.")
        return None

# Example Usage
public_key, private_key = asymmetricGenerateKeys(512)  # Generate RSA keys
print(f"Public Key: {public_key}")
print(f"Private Key: {private_key}")

message = "Hello, RSA with Base64!"
ciphertext = asymmetricEncrypt(message, public_key)
print(f"Encrypted (Base64): {ciphertext}")

decrypted_message = asymmetricDecrypt(ciphertext, private_key)
if decrypted_message:
    print(f"Decrypted Message: {decrypted_message}")
else:
    print("Decryption failed!")


Public Key: (65537, 3480102563014360870766006923750692351602204582731668303987728837843944203997428233350651080849840253974025519067580331989059572510718997258021623363073466068779892582625356983618475156179829851654187821592627105381174656319061842201483381537063822293231596693241871655392910108043450996566253826282527627447)
Private Key: (2357858829447894559479729090976120244872250638372740096464090873655868509240548176533392739108830993136833469996456497570993670723308293563146530085151150402103922787718741614201951116985349743608609057225102135368573114154477284211988155354661269826069011872315341516210188965672713053326782693629193663257, 3480102563014360870766006923750692351602204582731668303987728837843944203997428233350651080849840253974025519067580331989059572510718997258021623363073466068779892582625356983618475156179829851654187821592627105381174656319061842201483381537063822293231596693241871655392910108043450996566253826282527627447)
Encrypted (Base64): MjA1NzAzNzg1NTkwNzc

In [ ]:
import base64
def symmetricGenerateKeys(bits=512):
    return str(random.getrandbits(bits))

def symmetricEncrypt(kood, võti):
    """Encrypt a message using XOR and Base64 encoding."""
    packet_size = len(võti)
    if packet_size > 1:
        kood = [kood[i:i+packet_size] for i in range(0, len(kood), packet_size)]
    else:
        kood = [kood]

    vastus = "".join([chr(ord(c) ^ ord(võti[i])) for packet in kood for i, c in enumerate(packet)])
    return base64.b64encode(vastus.encode()).decode()

def symmetricDecrypt(kood, võti):
    """Decrypt a message that was encrypted using the encrypt function."""
    # Base64 decode first
    kood = base64.b64decode(kood).decode()

    packet_size = len(võti)
    if packet_size > 1:
        kood = [kood[i:i+packet_size] for i in range(0, len(kood), packet_size)]
    else:
        kood = [kood]

    # XOR Decryption (same as encryption)
    vastus = "".join([chr(ord(c) ^ ord(võti[i])) for packet in kood for i, c in enumerate(packet)])
    return vastus

# Example Usage
sõnum = "test"
võti = symmetricGenerateKeys(len(sõnum))
print(f"Võti: {võti}")

encrypted_once = symmetricEncrypt(sõnum, võti)
encrypted_twice = symmetricEncrypt(encrypted_once, võti)

# Decryption Process
decrypted_once = symmetricDecrypt(encrypted_twice, võti)
decrypted_twice = symmetricDecrypt(decrypted_once, võti)

print(f"Encrypted Once: {encrypted_once}")
print(f"Encrypted Twice: {encrypted_twice}")
print(f"Decrypted Once: {decrypted_once}")
print(f"Decrypted Twice (Original): {decrypted_twice}")


Võti: 15
Encrypted Once: RVBCQQ==
Encrypted Twice: Y2NzdmBkDAg=
Decrypted Once: RVBCQQ==
Decrypted Twice (Original): test


In [ ]:
class AllkirjadeAndmebaas:
    def __init__(self):
        self.allkirjadPrivate = {}  # {nimi: (allkiri, private)}

    def kontrolliAllkirja(self, name, cypher):
        (allkiri, private) = self.allkirjadPrivate[name]
        # The cypher should be the ENCRYPTED signature, not the signature itself
        decrypted_cypher = asymmetricDecrypt(cypher, private)
        # Compare the decrypted cypher with the stored signature
        if decrypted_cypher == allkiri:
            return True
        else:
            return False

    def looAllkirja(self, nimi, bits=512):
        if nimi in self.allkirjadPrivate:
            return None
        public_key, private_key = asymmetricGenerateKeys(512)
        # Change random.str to str(random.getrandbits(bits))
        cypher = str(random.getrandbits(bits))  # Convert random bits to string using str()
        self.allkirjadPrivate[nimi] = (cypher, private_key)
        return public_key, cypher

In [ ]:
class Klient:
    def __init__(self, nimi, AllkirjadeAndmebaas_):
        self.nimi = nimi
        self.allkiri = AllkirjadeAndmebaas_.looAllkirja(nimi)
        self.AllkirjadeAndmebaas_ = AllkirjadeAndmebaas_

        if self.allkiri is None:
            raise Exception("See nimi on juba kasutusel")

        self.uhendusAllkiri = None
        self.uhendusSessionVoti = None

    def looUhendus(self, klient, nimi):
        # Encrypt the signature before sending
        encrypted_signature = asymmetricEncrypt(self.allkiri[0], klient.allkiri[0])
        klientAllkiri = klient.vastaUhendus(nimi, encrypted_signature)
        if klientAllkiri is None or not self.AllkirjadeAndmebaas_.kontrolliAllkirja(nimi, klientAllkiri):
            return None

        print("Loodud ühendus: vahetame session võtit")
        self.uhendusSessionVoti = symmetricGenerateKeys()
        klient.lopetaUhendus(asymmetricEncrypt(self.uhendusSessionVoti, klientAllkiri))

    def lopetaUhendus(self, cytherSessionVoti):
        self.uhendusSessionVoti = asymmetricDecrypt(cytherSessionVoti, self.allkiri)
        self.uhendusAllkiri = None

    def vastaUhendus(self, nimi, allkiri):
        if not self.AllkirjadeAndmebaas_.kontrolliAllkirja(nimi, allkiri):
            return None
        # Encrypt the signature before sending
        encrypted_signature = asymmetricEncrypt(self.allkiri[0], allkiri)
        return encrypted_signature

    def send(self, klient, sonum):
        if self.uhendusSessionVoti is None:
            return None
        klient.sonumPuua(symmetricEncrypt(sonum, self.uhendusSessionVoti))

    def sonumPuua(self, cypher):
        print(f"{self.nimi} received: {symmetricDecrypt(cypher, self.uhendusSessionVoti)}")


In [ ]:
db = AllkirjadeAndmebaas()

# Create two clients
klient1 = Klient("Alice", db)
klient2 = Klient("Bob", db)

# Step 1: Establish a connection between Alice and Bob
print("\n[TEST] Establishing connection...")
klient1.looUhendus(klient2, "Bob")

# Step 2: Check if the connection was successful
if klient1.uhendusSessionVoti and klient2.uhendusSessionVoti:
    print("[SUCCESS] Connection established!\n")
else:
    print("[FAIL] Connection failed!\n")

# Step 3: Alice sends a message to Bob
print("[TEST] Alice sends a message to Bob...")
klient1.send(klient2, "Hello, Bob!")

# Step 4: Bob sends a message to Alice
print("[TEST] Bob sends a message to Alice...")
klient2.send(klient1, "Hi Alice! Nice to meet you.")

# Step 5: Test invalid connection attempt (Duplicate name)
print("\n[TEST] Attempting to create a new client with Alice's name...")
try:
    klient3 = Klient("Alice", db)
except Exception as e:
    print(f"[EXPECTED ERROR] {e}")

print("\n[TEST COMPLETE]")


[TEST] Establishing connection...


AttributeError: 'tuple' object has no attribute 'encode'